In [1]:
import os, serial, time
from numpy import *

# Communication functions

In [79]:
def updateLed( sObj, dat=None, channel=0 ):
    if dat is None:
        nLeds = 1024
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    elif type(dat) is int:
        nLeds = dat
        ledDat = bytearray( os.urandom( nLeds*3 ) )
    else:
        nLeds = len(dat)
        ledDat = bytearray( dat )
    sDat = bytes("LED {0} {1}\n".format(channel, len(ledDat)), "utf8") + ledDat
    sObj.write( sDat )

# Open serial port connection

In [111]:
try:
    s.close()
except:
    pass

In [112]:
s = serial.Serial( "/dev/ttyACM1", timeout=1 )

# Get ID and Software Version

In [113]:
s.read_all()          #Clear receive buffer
s.write(b"\n*IDN?\n") #First \n clears send buffer
s.read_until()

b'ID:MB:V0.0\n'

# Get Switch state

In [114]:
#%%timeit   #1000 loops, best of 3: 1.78 ms per loop
s.write(b"SW?\n")
s.read_until()

b'SW:00000000000000000000000000000000000000000000000000000000000000000000000000000000\n'

# Setup low speed WS2811 LED strand

In [77]:
s.write(b"LEC 0 1601000\n")

14

Datasheet spec. would be **1600000 Hz**, but my LED strand glitches like hell with that setting. **1601000 Hz** works perfectly though.

# Setup high speed WS2812 LED strip

In [90]:
s.write(b"LEC 0 3200000\n")

14

# Benchmark LED throughput (worst case)

In [134]:
%%timeit
x = array( ones(1024*3), dtype=uint8)
updateLed(s, x, 0)
updateLed(s, x, 1)
updateLed(s, x, 2)
# [Only USB comm.] 10 loops, best of 100: 36.0 ms per loop (28 Hz)
# [USB + SPI send] 10 loops, best of 100: 45.5 ms per loop (22 Hz)

10 loops, best of 3: 55.4 ms per loop


It looks like the USB communication is the bottleneck at the moment. Note that the firmware can transmit on several channels simultaneously but will block if a channel is updated, which has not finished transmitting yet.

**TLDR:** If you need > 30 Hz refresh rate, do not connect more than 512 LEDs per string

# Play with LEDs

### Glitch test

Do many sets of the same values. LEDs should not flicker at all!

In [132]:
x = zeros( 59*3, dtype=uint8 )
x[0:6] = 1
x[-6:] = 1
for i in range(1000):
    updateLed( s, x, 0 )

### Turn ON one color after another

In [ ]:
while(True):
    x = zeros( 59*3, dtype=uint8 )
    for i in range(len(x)):
        x[i] = 1
        updateLed(s, x, 0)
        time.sleep(0.03)

### All LEDs same color

In [978]:
x = array( [0, 0, 0]*59, dtype=uint8 )
updateLed( s, x, 0 )

### Gamma corrected fade-UP

In [133]:
# Precompute quadratic brightness values
bVals = array( arange(33)**2/4, dtype=uint8 )
bVals[-1] = 255

In [987]:
for bVal in bVals:
    x = array( [bVal, bVal, bVal]*5, dtype=uint8 )
    updateLed( s, x, 0 )
    time.sleep(0.03)

### Each LED a random color

In [ ]:
# Set it once
x = array(random.randint(0,255,58*3),dtype=uint8)
updateLed(s, x, 0)

In [750]:
# Set it in a loop
while True:
    x = array(random.randint(0,255,58*3),dtype=uint8)
    updateLed( s, x, 0 )
    time.sleep(0.3)

KeyboardInterrupt: 